In [1]:
!pip install -q openai langchain faiss-cpu pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.1 MB/s eta 0:00:00


In [2]:
!pip install -q fuzzywuzzy python-Levenshtein openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.9 MB/s eta 0:00:00


In [10]:
from google.colab import files
uploaded = files.upload()


Saving ICD 10 - Codes_Jan2025.xlsx to ICD 10 - Codes_Jan2025 (1).xlsx
Saving OPCS code - opcs4-8_toce_analysis_nov_2016_v1_0.xls to OPCS code - opcs4-8_toce_analysis_nov_2016_v1_0 (1).xls


In [12]:
import pandas as pd


# Load ICD-10 codes
icd = pd.read_excel("ICD 10 - Codes_Jan2025.xlsx")
icd = icd.dropna()

# Load OPCS procedure codes
opcs = pd.read_excel("OPCS code - opcs4-8_toce_analysis_nov_2016_v1_0.xls")
opcs = opcs.dropna()


In [14]:

# ICD-10
icd["CombinedText"] = (
    "ICD Code: " + icd.iloc[:, 0].astype(str) +  # First col usually the code
    " | " + icd.iloc[:, 1].astype(str)           # Second col is diagnosis
)

# OPCS
opcs["CombinedText"] = (
    "OPCS Code: " + opcs.iloc[:, 0].astype(str) +
    " | " + opcs.iloc[:, 1].astype(str)
)


In [17]:
merged_df = pd.concat([
    icd[["CombinedText"]],
    opcs[["CombinedText"]]
], ignore_index=True)

print("✅ Combined dataset shape:", merged_df.shape)
merged_df.head(2)


✅ Combined dataset shape: (4898, 1)


,CombinedText
0,ICD Code: A7741 | Ehrlichiosis chaffeensis [E....
1,ICD Code: A7982 | Anaplasmosis [A. phagocytoph...


In [18]:
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(merged_df, page_content_column="CombinedText")
docs = loader.load()


In [19]:
!pip install -q faiss-cpu langchain langchain-community sentence-transformers openrouter httpx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.3 MB/s eta 0:00:00


In [21]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Create embedding using a free model (all local)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector DB
db = FAISS.from_documents(docs, embeddings)


<ipython-input-21-3f1769974c31>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
import os
from langchain.chat_models import ChatOpenAI

# Set OpenRouter API credentials
os.environ["OPENAI_API_KEY"] = "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"  # Paste your key here
os.environ["OPENAI_BASE_URL"] = "https://openrouter.ai/api/v1"  # Required base URL override

# Load Mistral via OpenRouter
llm = ChatOpenAI(
    model="mistralai/mistral-7b-instruct",  # Or use another from OpenRouter
    base_url="https://openrouter.ai/api/v1",
    temperature=0.3
)


In [26]:
from langchain.chains import RetrievalQA

retriever = db.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)


In [28]:
query = input("Ask your question: ")
response = qa_chain.run(query)

print("🧠 Query:", query)
print("🧠 Answer:", response)


Ask your question: what is code A0101
🧠 Query: what is code A0101
🧠 Answer:  I'm sorry for any confusion, but the provided OPCS codes (Current Procedural Terminology, CPT, or ICD-10 codes) do not include A0101. The codes provided are V01.9 (Unspecified encounter for follow-up of a non-neonatal patient) and A02.9 (Injection, other and unspecified, not elsewhere classified). If you have a different set of codes that includes A0101, please provide them for a more accurate response.
